In [1]:
%load_ext autoreload
%autoreload 2
from spacy_stanfordnlp import StanfordNLPLanguage
import stanfordnlp
from copy import deepcopy
from spacy_conll import ConllFormatter
from pathlib import Path
from qa2nli.qa_readers.race import read_data as race_read_data
from qa2nli.qa_readers.race import process_samples_lazy
from rule import Question, AnswerSpan
from conllu import parse
from sacremoses import MosesTokenizer, MosesDetokenizer
import tqdm
import multiprocessing as mp
from typing import *

In [2]:
#stanfordnlp.download('en')
# Config
input_path = Path('dev_has_following.json')
output_path = Path('.data/RACE/converted_dev.json')

In [3]:
snlp = stanfordnlp.Pipeline(lang='en')
nlp = StanfordNLPLanguage(snlp)
conllformatter = ConllFormatter(nlp)
nlp.add_pipe(conllformatter, last=True)
detokenizer = MosesDetokenizer()

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/dhruv/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/Users/dhruv/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/Users/dhruv/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/dhruv/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/Users/dhruv/stanfordnlp_resources/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/Users/dhruv/stanfordnlp_resources/en_ewt_models/en_

In [4]:
# load data
with open(input_path) as f:
    samples = json.load(f)

In [5]:
import warnings
warnings.filterwarnings("ignore")
def convert(qa_samples: List[Dict]):
    total_questions=0
    num_converted_questions = 0
    num_nli_samples = 0
    invalid_questions = 0
    invalid_options = 0 
    failed_questions = []
    nli_samples = []
    failed_cases = []
    for qa_sample in tqdm.tqdm(qa_samples):
        total_questions+=1
        q_doc = nlp(qa_sample['question'])
        a_doc = nlp(qa_sample['answer'])
        q_conll_dict = parse(q_doc._.conll_str)[0].tokens
        a_conll_dict = parse(a_doc._.conll_str)[0].tokens
        positive_sample = {}
        q = Question(deepcopy(q_conll_dict))
        if not q.isvalid:
            invalid_questions+=1
            failed_cases.append(qa_sample['id'])
            continue
        a = AnswerSpan(a_conll_dict)
        if not a.isvalid:
            failed_cases.append(qa_sample['id'] + '_1')
            invalid_options+=1
        else:
            q.insert_answer_default(a)
            hypo = detokenizer.detokenize(q.format_declr(), return_str=True)
            positive_sample['premise'] = qa_sample['article']
            positive_sample['hypothesis'] = hypo
            positive_sample['label'] = 1
            positive_sample['id'] = qa_sample['id'] + '_1'
        if positive_sample:
            nli_samples.append(positive_sample)
        for i, opt in enumerate(qa_sample['other_options']):
            q = Question(deepcopy(q_conll_dict))
            o_conll_dict = parse(nlp(opt)._.conll_str)[0].tokens
            o = AnswerSpan(o_conll_dict)
            negative_sample = {}
            if not o.isvalid:
                invalid_options+=1
                failed_cases.append(qa_sample['id'] + '_' + str(i+1))
                continue
            else:
                q.insert_answer_default(o)
                negative_sample['premise'] = qa_sample['article']
                negative_sample['hypothesis'] = detokenizer.detokenize(q.format_declr(), return_str=True)
                negative_sample['label'] = 0
                negative_sample['id'] = qa_sample['id'] + '_' + str(i+1)
            if negative_sample:
                nli_samples.append(negative_sample)
        num_converted_questions+=1
        return {'nli_samples': nli_samples, 'failed_questions': failed_questions, 'total_questions': total_questions,
               'failed_cases': failed_cases}




In [29]:
def convert2(qa_sample: Dict):
    q_doc = nlp(qa_sample['question'])
    a_doc = nlp(qa_sample['option'])
    q_conll_dict = parse(q_doc._.conll_str)[0].tokens
    a_conll_dict = parse(a_doc._.conll_str)[0].tokens
    qa_sample.update({'valid_question': False, 'valid_option': False, 'conversion_success': False})
    q = Question(deepcopy(q_conll_dict))
    if not q.isvalid:
        qa_sample['hypothesis'] = ""
        return qa_sample
    else:
        qa_sample['valid_question'] = True
    a = AnswerSpan(a_conll_dict)
    if not a.isvalid:
        qa_sample['hypothesis'] = ""
        return qa_sample
    else:
        q.insert_answer_default(a)
        qa_sample['hypothesis'] = detokenizer.detokenize(q.format_declr(), return_str=True)
    return qa_sample

In [7]:
converted = convert2(samples[0])

> /Users/dhruv/UnsyncedDocuments/IBM_project/qa-to-nli/rule.py(124)__init__()
-> if self.isvalid:
(Pdb) c
> /Users/dhruv/UnsyncedDocuments/IBM_project/qa-to-nli/rule.py(208)_wh_true_false()
-> neg = self._get_children(root, ['advmod', 'neg'], 'left')
(Pdb) c
> /Users/dhruv/UnsyncedDocuments/IBM_project/qa-to-nli/rule.py(464)get_answer_pos()
-> if self._answer_pos is not None:
(Pdb) c


In [8]:
converted

{'id': 'high939.txt_0_B',
 'premise': "All Eskimos live most of their lives close to sea or fresh water. They may follow game inland for several hundred miles, but they always return to the shores of rivers, lakes, or seas. Eskimo land has a bare look. Large rocks, pebbles, and sand cover much of the surface. Plants called lichen   grow on the rocks, and where there is enough soil, grass, flowers, and even small bushes manage to live. No trees can grow on Eskimo land, so geographers sometimes call it the Arctic plains. There are some animals in Eskimo land, such as rabbits, which eat the plants. Other animals, like the white fox and the gray wolf, eat the rabbits. The Eskimo is a meat-eater, too, and may even eat a wolf when food is scarce.\nThe Eskimo year has two main parts: a long, cold winter and a short, cool summer. Spring and fall are almost too short to be noticed. Summer is the best time, as food is usually plentiful. But it is also the time when Eskimos are very busy. Winter 

In [32]:
convert2(samples[209])

{'id': 'high4783.txt_1_C',
 'premise': 'An old problem is getting new attention in the United States--bullying. Recent cases included the tragic case of a 15-year-old girl whose family moved from Ireland. She hanged herself in Massachusetts following months of bullying. Her parents criticized her school for failing to protect her. Officials have brought criminal charges against several teenagers.\nJudy Kuczynski is president of an anti-bullying group called Bully Police USA. Her daughter Tina was also the victim of severe bullying starting in middle school in the state of Minnesota. She said,"Our daughter was a very outgoing child. She was involved in all kinds of things and had lots of friends. And over a period of time her grades fell completely. She started having health issues. She couldn\'t sleep. She wasn\'t eating. She had terrible stomach pains and didn\'t want to go to school."\nBullying is defined as negative behavior repeated over time against the same person. It can involve

In [13]:
converted = []
with mp.Pool(processes=3) as p:
        with tqdm.tqdm(total=num_processed_samples) as pbar:
            for i, _ in enumerate(p.imap_unordered(convert2, processed_samples)):
                converted.append(_)
                pbar.update()

  4%|▍         | 850/19548 [01:16<27:58, 11.14it/s] Process ForkPoolWorker-10:



KeyboardInterrupt: 